In [142]:
#####ベイジアン確率的ブロックモデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

In [143]:
####データの発生####
##データの設定
N = 5000   #ユーザー数
K = 2000   #アイテム数
seg_u = 8   #ユーザーセグメント数
seg_i = 6   #アイテムセグメント数

##セグメントとパラメータを生成
#ユーザーセグメントを生成
alpha01 = np.repeat(100, seg_u)
pi01 = np.random.dirichlet(alpha01, 1).reshape(seg_u)
z01 = np.random.multinomial(1, pi01, N)
z01_vec = np.dot(z01, range(seg_u))
z1_cnt = np.sum(z01, axis=0)
print(np.mean(z01, axis=0))

#アイテムセグメントを生成
alpha02 = np.repeat(100, seg_i)
pi02 = np.random.dirichlet(alpha02, 1).reshape(seg_i)
z02 = np.random.multinomial(1, pi02, K)
z02_vec = np.dot(z02, range(seg_i))
z2_cnt = np.sum(z02, axis=0)
print(np.mean(z02, axis=0))

#観測変数のパラメータの設定
#ユーザーセグメント×アイテムセグメントのベータ事前分布のパラメータを生成
theta0 = np.random.beta(1.0, 6.5, seg_u*seg_i).reshape((seg_u, seg_i))

[0.1194 0.1192 0.1226 0.132  0.1222 0.1306 0.1106 0.1434]
[0.1765 0.1565 0.1775 0.1445 0.166  0.179 ]


In [188]:
##ベルヌーイ分布から観測行列を生成
Data = np.zeros((N, K))
for i in range(seg_u):
    print(i)
    index_z01 = ((z01_vec==i) + 0) * range(1, N+1)
    a = index_z01[index_z01!=0]-1
    
    for j in range(seg_i):
        n = z1_cnt[i] * z2_cnt[j]
        index_z02 = ((z02_vec==j) + 0) * range(1, K+1)
        b = index_z02[index_z02!=0]-1
        y = np.random.binomial(1, theta0[i, j], n).reshape((z1_cnt[i], z2_cnt[j]))
        
        for k in range(z2_cnt[j]):
            Data[z01_vec==i, b[k]] = y[:, k]

0
1
2
3
4
5
6
7


array([ True, False, False, ..., False, False, False])

316232

In [116]:
Data[0, 0]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (680,) (333,) 

array([False, False, False, ..., False, False, False])

In [135]:
Data[a, :] = 1

In [137]:
Data[a, :]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])